In [ ]:
import numpy as np
import hades.aero.ShockWave  as sw
import hades.aero.unsteady1D as uq
import hades.aero.unsteady_inflow as inflow
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 14 ; plt.rcParams['axes.grid'] = True ; plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['lines.linewidth'] = 1.5 ; plt.rcParams['legend.fontsize'] = 12
#print plt.rcParams.keys()
# for slides
#plt.rcParams['font.size'] = 24 ; plt.rcParams['grid.linewidth'] = 2 ; plt.rcParams['lines.linewidth'] = 4

In [ ]:
M0 = 0.3
qi = uq.unsteady_state(rho=1., u=M0*np.sqrt(1.4), p=1.)
rtti = qi.rTtot()
# defintion du probleme instationnaire en entree, qi est etat initial donc qR
inlet = inflow.inflow_pb(qi)

taux_tt =1.

pi_range = np.arange(1., 5., .05) ; n = pi_range.size
mf0 = np.ones(n)
mf1 = np.ones(n)
mf2 = np.ones(n)
mff = np.ones(n)
mffb = np.ones(n)
taux_ps = np.ones(n)
Ms = np.ones(n)
M1 = np.ones(n)
M2 = np.ones(n)

for i in range(n):
    # calculs des etats intermediaires, resultat de Pt impose
    inlet.solve_with_pt0_rtt0(pi_range[i]*qi.Ptot(), taux_tt*rtti)
    qf  = qi.copy()
    qfb = qi.copy()
    # Qf est etat final qui satisfait Pt amont et p0 aval de etat initial
    qf.compute_from_pt_rtt_p(pi_range[i]*qi.Ptot(), taux_tt*rtti, qi.p)
    qfb.compute_from_pt_rtt_M(pi_range[i]*qi.Ptot(), taux_tt*rtti, M0)
    # extraction des resultats
    mf0[i]  = inlet._qR.massflow()
    mf1[i]  = inlet._qstarR.massflow()
    mf2[i]  = inlet._qstarL.massflow()
    mff[i]  = qf.massflow()
    mffb[i] = qfb.massflow()
    taux_ps[i] = inlet._qstarR.p / inlet._qR.p
    Ms[i] = sw.Mn_Ps_ratio(taux_ps[i])
    M1[i] = inlet._qstarR.Mach()
    M2[i] = inlet._qstarL.Mach()    
#
fig, ax = plt.subplots(1,3, figsize=[18,5])
#
ax[0].plot(pi_range, mf0, pi_range, mf1, pi_range, mf2, pi_range, mff, pi_range, mffb)
ax[0].set(xlabel='taux $P_t$', ylabel='massflow')
ax[0].legend(('mf0', 'mf1', 'mf2', 'mff', 'mffb'), loc='best')
#
ax[1].plot(pi_range, Ms, pi_range, taux_ps, pi_range, M1, pi_range, M2)
ax[1].set(xlabel='taux $P_t$', ylabel='$M_s$, $\Pi$, $M_1$, $M_2$')
ax[1].legend(('$M_s$', '$\pi_{sh}$', '$M_1$', '$M_2$'), loc='best')
#
ax[2].plot(pi_range, (mf1-mf0)/(mff-mf0), pi_range, (mf2-mf0)/(mff-mf0), pi_range, (mf1-mf0)/(mffb-mf0), pi_range, (mf2-mf0)/(mffb-mf0))
ax[2].set(xlabel='taux $P_t$', ylabel='$\dot{m}$ normalise')
ax[2].legend(('1f', '2f', '1fb', '2fb'), loc='best')


In [ ]:
#print plt.rcParams